In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import statsmodels.formula.api as smf

In [11]:
file = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\reg_TA_PA_DL_old.csv"
df1 = pd.read_csv(file)
len(df1)

2808

In [12]:
#in df remove all rows that contain a NaN
df1 = df1.dropna()
df1.reset_index(drop=True, inplace=True)
df1.rename(columns={'time_x':'time'}, inplace=True)


df1['admin1'] = df1['admin1'].str.replace('Lower Shabelle', 'Lower_Shabelle')
df1['admin1'] = df1['admin1'].str.replace('Middle Shabelle', 'Middle_Shabelle')
df1['admin1'] = df1['admin1'].str.replace('Lower Juba', 'Lower_Juba')
df1['admin1'] = df1['admin1'].str.replace('Middle Juba', 'Middle_Juba')
df1['admin1'] = df1['admin1'].str.replace('Woqooyi Galbeed', 'Woqooyi_Galbeed')


In [13]:
df1 = df1.dropna()

In [14]:
#df1.to_csv('tpc_prot.csv', index=False)

In [15]:
df=df1

In [23]:
#Create the dummy variables, one for each country
df_dummies = pd.get_dummies(df['admin1'])
df_with_dummies = df.join(df_dummies)

#Create the dummy variables, one for each month
df['month'] = pd.DatetimeIndex(df['time']).month_name()
df_dummies_m = pd.get_dummies(df['month'])
#
df_with_dummies = df_with_dummies.join(df_dummies_m)
df_with_dummies['month'] = pd.DatetimeIndex(df['time']).month

#Create the dummy variables, one for each for each country-month pair
df_dummies_mr = pd.get_dummies(df['admin1'] + df['month'])
df_with_dummies = df_with_dummies.join(df_dummies_mr)
df_with_dummies = df_with_dummies.replace({True: 1, False: 0})

In [34]:
df_with_dummies['TAm']=df_with_dummies['TA'].apply(lambda x: x+0.5)

In [38]:
y_var_name = 'conflicts'
X_var_names = ['TA','PA','DL']

In [39]:
unit_names = df['admin1'].unique().tolist()
unit_names.sort()
unit_names_t = df['month'].unique().tolist()
unit_names_mr = (df['admin1'] + df['month']).unique().tolist()

lsdv_expr = y_var_name + ' ~ '
i = 0
for X_var_name in X_var_names:
    if i > 0:
        lsdv_expr = lsdv_expr + ' + ' + X_var_name
    else:
        lsdv_expr = lsdv_expr + X_var_name
    i = i + 1
for dummy_name in unit_names[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name
for dummy_name_t in unit_names_t[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name_t
for dummy_name_mr in unit_names_mr[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name_mr
 
print('Regression expression for OLS with dummies=' + lsdv_expr)

Regression expression for OLS with dummies=conflicts ~ TA + PA + DL + Awdal + Bakool + Banadir + Bay + Galgaduud + Gedo + Hiraan + Lower_Juba + Lower_Shabelle + Middle_Juba + Middle_Shabelle + Mudug + Sanaag + Sool + Togdheer + March + April + May + June + July + August + September + October + November + December + January + AwdalMarch + AwdalApril + AwdalMay + AwdalJune + AwdalJuly + AwdalAugust + AwdalSeptember + AwdalOctober + AwdalNovember + AwdalDecember + AwdalJanuary + AwdalFebruary + BakoolMarch + BakoolApril + BakoolMay + BakoolJune + BakoolJuly + BakoolAugust + BakoolSeptember + BakoolOctober + BakoolNovember + BakoolDecember + BakoolJanuary + BakoolFebruary + BanadirMarch + BanadirApril + BanadirMay + BanadirJune + BanadirJuly + BanadirAugust + BanadirSeptember + BanadirOctober + BanadirNovember + BanadirDecember + BanadirJanuary + BanadirFebruary + BayMarch + BayApril + BayMay + BayJune + BayJuly + BayAugust + BaySeptember + BayOctober + BayNovember + BayDecember + BayJanua

In [40]:
lsdv_model = smf.ols(formula=lsdv_expr, data=df_with_dummies)
lsdv_model_results = lsdv_model.fit()
print(lsdv_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:              conflicts   R-squared:                       0.299
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     4.980
Date:                Thu, 20 Jul 2023   Prob (F-statistic):           3.89e-77
Time:                        10:26:21   Log-Likelihood:                -9144.2
No. Observations:                2347   AIC:                         1.866e+04
Df Residuals:                    2161   BIC:                         1.973e+04
Df Model:                         185                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [41]:
lsdv_model = smf.ols(formula=lsdv_expr, data=df_with_dummies)
lsdv_model_results = lsdv_model.fit()
print(lsdv_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:              conflicts   R-squared:                       0.299
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     4.980
Date:                Thu, 20 Jul 2023   Prob (F-statistic):           3.89e-77
Time:                        10:26:21   Log-Likelihood:                -9144.2
No. Observations:                2347   AIC:                         1.866e+04
Df Residuals:                    2161   BIC:                         1.973e+04
Df Model:                         185                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [42]:
#with open('ols_summary_2.txt', 'w') as f:
   # f.write(lsdv_model_results.summary().as_text())